# Parsimonious Learning-Augmented Caching

Analyze the results of that paper. Mainly that table here:

| Algorithms | Mean Predictions | σ=0 | σ=2 | σ=4 | σ=6 |
|------------|------------------|-----|-----|-----|-----|
| RandomMarker | 3.14 | 3.14 | 3.14 | 3.14 | 3.14 |
| LRU | 2.86 | 2.86 | 2.86 | 2.86 | 2.86 |
| BlindOracle | 1.92 | 1.00 | 1.02 | 3.92 | 4.15 |
| LVMarker | 2.49 | 1.77 | 1.81 | 2.94 | 3.11 |
| RohatgiMarker | 2.54 | 1.77 | 1.83 | 3.15 | 3.29 |
| RobustOracle | 4.29 | 1.80 | 1.83 | 4.48 | 4.51 |
| AdaptiveQuery-2 | 2.91 | 2.46 | 2.46 | 2.52 | 2.65 |
| AdaptiveQuery-4 | 2.71 | 2.07 | 2.07 | 2.20 | 2.49 |
| AdaptiveQuery-8 | 2.59 | 1.86 | 1.86 | 2.07 | 2.54 |

In [2]:
import pandas as pd

Loading BokehJS ...

In [17]:
df = pd.read_csv('tripdata/201808-citibike-tripdata.csv').dropna()
df['start station id'] = df['start station id'].astype(int)
df[['starttime', 'stoptime', 'start station id', 'end station id']].head()

,starttime,stoptime,start station id,end station id
0,2018-08-01 00:00:07.3210,2018-08-01 00:11:28.9920,3162,3383.0
1,2018-08-01 00:00:19.7480,2018-08-01 00:10:45.0290,3260,2012.0
2,2018-08-01 00:00:21.1750,2018-08-01 00:22:20.6370,403,285.0
3,2018-08-01 00:00:26.4700,2018-08-01 00:04:06.8190,3637,399.0
4,2018-08-01 00:00:30.2910,2018-08-01 00:07:09.2810,3662,3517.0


In [21]:
df.sort_values(by='starttime').reset_index(drop=True).describe().transpose()

,count,mean,std,min,25%,50%,75%,max
tripduration,1975789.0,1010.754825,11305.846867,61.000000,374.000000,640.000000,1129.000000,3.095079e+06
start station id,1975789.0,1627.431002,1449.333004,72.000000,382.000000,514.000000,3258.000000,3.705000e+03
start station latitude,1975789.0,40.737313,0.030810,40.646538,40.717452,40.737815,40.758985,4.081439e+01
start station longitude,1975789.0,-73.982474,0.019622,-74.025353,-73.995960,-73.985649,-73.971789,-7.390774e+01
end station id,1975789.0,1622.044900,1449.196796,72.000000,382.000000,513.000000,3258.000000,3.705000e+03
end station latitude,1975789.0,40.736955,0.030669,40.646538,40.717400,40.737262,40.757632,4.081439e+01
end station longitude,1975789.0,-73.982616,0.019724,-74.063784,-73.996010,-73.986022,-73.971879,-7.390774e+01
bikeid,1975789.0,26185.662380,6233.147262,14529.000000,19951.000000,27745.000000,31502.000000,3.483900e+04
birth year,1975789.0,1979.599159,11.706486,1885.000000,1969.000000,1982.000000,1989.000000,2.002000e+03
gender,1975789.0,1.136640,0.570710,0.000000,1.000000,1.000000,1.000000,2.000000e+00


In [15]:
df.sort_values(by='starttime').reset_index()['start station id']

0          315
1         3224
2          386
3          466
4          438
          ... 
718989    3556
718990     483
718991     449
718992    3146
718993     310
Name: start station id, Length: 718994, dtype: int64

In [22]:
def extract_station_ids(month):
    df = pd.read_csv(f"tripdata/{month}-citibike-tripdata.csv")[['starttime', 'start station id']].dropna()
    df['start station id'] = df['start station id'].astype(int)
    df.sort_values(by='starttime', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

def store_station_ids(month):
    df = extract_station_ids(month)[['start station id']]
    df.to_csv(f"start_stations/{month}-citibike-stations.csv", index=False, header=False)

In [23]:
# Uncomment to recompute all station ids
# for month in range(1, 13):
#     store_station_ids(f"2018{month:02d}")

In [24]:
def count_distinct_stations(month):
    df = pd.read_csv(f"start_stations/{month}-citibike-stations.csv", header=None)
    return df[0].head(25000).nunique()

def count_distinct_stations_from_tripdata(month):
    df = pd.read_csv(f"tripdata/{month}-citibike-tripdata.csv")
    return df['start station id'].head(25000).nunique()

for month in range(1, 13):
    res = count_distinct_stations(f"2018{month:02d}")
    res_2 = count_distinct_stations_from_tripdata(f"2018{month:02d}")
    print(f"2018-{month:02d}: {res} vs. {res_2}")

2018-01: 727 vs. 19
2018-02: 741 vs. 18
2018-03: 748 vs. 16
2018-04: 747 vs. 13
2018-05: 742 vs. 9
2018-06: 733 vs. 8
2018-07: 733 vs. 9
2018-08: 748 vs. 748
2018-09: 753 vs. 753
2018-10: 753 vs. 752
2018-11: 740 vs. 739
2018-12: 745 vs. 745
